# Learning Geocoding
### Understanding how to convert addresses to latitude/longitude coordinates using Google Maps API
Learned via tutorial at https://medium.datadriveninvestor.com/turning-addresses-into-coordinates-de6198b22905

#### First, intall the googlemaps library:

`pip install googlemaps`

OR

`conda install -c conda-forge googlemaps`

#### Imports, etc. 

Note: must first create a Google Cloud account to get an API key (must add create billing account and add credit card, but small-time use is not charged)

In [1]:
import googlemaps
from env import api_key
gmaps = googlemaps.Client(key=api_key)

#### Passing an address to the google maps geocode() function returns a json as follows:
(location lat/lng coordinates are found at `json[0]['geometry']['location']`

In [2]:
gmaps.geocode('600 Navarro St #600, San Antonio, TX 78205')

[{'address_components': [{'long_name': '600',
    'short_name': '600',
    'types': ['subpremise']},
   {'long_name': '600', 'short_name': '600', 'types': ['street_number']},
   {'long_name': 'Navarro Street',
    'short_name': 'Navarro St',
    'types': ['route']},
   {'long_name': 'Downtown',
    'short_name': 'Downtown',
    'types': ['neighborhood', 'political']},
   {'long_name': 'San Antonio',
    'short_name': 'San Antonio',
    'types': ['locality', 'political']},
   {'long_name': 'Bexar County',
    'short_name': 'Bexar County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Texas',
    'short_name': 'TX',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '78205', 'short_name': '78205', 'types': ['postal_code']}],
  'formatted_address': '600 Navarro St #600, San Antonio, TX 78205, USA',
  'geometry': {'location': {'lat': 29

#### We can define a function to accept an address and return a dictionary containig the lat/lng coordinates from the gmaps.geocode() result

In [3]:
def get_coords(address):
    geocode_result = gmaps.geocode(address)
    return geocode_result[0]['geometry']['location'] 

In [4]:
codeup_location = get_coords('600 Navarro St #600, San Antonio, TX 78205')
codeup_location

{'lat': 29.4267857, 'lng': -98.4895763}

Note: for real-world applications, we should enhance the get_coords() function to accept addresses in multiple formats (or define a function that will convert addresses to the appropriate format). We should also account for the possibility that the call to gmaps.geocode() will not return the expected result. 

## Incorporating it into a larger dataset

In [5]:
import numpy as np
import pandas as pd

#### Starting with a dataframe of address strings:

In [6]:
names = ['Codeup', 
         'Schilo\'s',
         'Bunz']

addresses = ['600 Navarro St #600, San Antonio, TX 78205',
             '424 E Commerce St, San Antonio, TX 78205',
             '122 E Houston St, San Antonio, TX 78205']

df = pd.DataFrame({'name': names, 'address': addresses})
df

,name,address
0,Codeup,"600 Navarro St #600, San Antonio, TX 78205"
1,Schilo's,"424 E Commerce St, San Antonio, TX 78205"
2,Bunz,"122 E Houston St, San Antonio, TX 78205"


#### Get the coordintes for each address:

In [7]:
# create a new column named 'coordinates' and populate it with the result of get_coords() for each address
df['coordinates'] = df.address.apply(get_coords)

In [8]:
df

,name,address,coordinates
0,Codeup,"600 Navarro St #600, San Antonio, TX 78205","{'lat': 29.4267857, 'lng': -98.4895763}"
1,Schilo's,"424 E Commerce St, San Antonio, TX 78205","{'lat': 29.4236082, 'lng': -98.4883196}"
2,Bunz,"122 E Houston St, San Antonio, TX 78205","{'lat': 29.4263142, 'lng': -98.4929289}"


#### Separate the coordinates into distinct values for latitude and longitude

In [9]:
# create separate columns for latitude/longitude
df['latitude'] = df.coordinates.apply(lambda x: x['lat'])
df['longitude'] = df.coordinates.apply(lambda x: x['lng'])
# drop coordinates column
df = df.drop(columns='coordinates')

In [10]:
df

,name,address,latitude,longitude
0,Codeup,"600 Navarro St #600, San Antonio, TX 78205",29.426786,-98.489576
1,Schilo's,"424 E Commerce St, San Antonio, TX 78205",29.423608,-98.488320
2,Bunz,"122 E Houston St, San Antonio, TX 78205",29.426314,-98.492929


In [12]:
df.to_csv('example_df.csv')